In [1]:
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import os
import scipy

In [2]:
data = "images"

In [3]:
# start by parsing the command line arguments 
my_batch_size = 40
my_epochs = 80
augment_data = '1'
fine_tune = '1'
batch_size = "100"
epochs = "80"
args = epochs + batch_size + fine_tune + augment_data
main_dir = "/tf"

h5modeloutput = 'model_b' + batch_size + '_e' + epochs + '_aug' + \
    augment_data + '_ft' + fine_tune + '.h5'

# Load weights pre-trained on the ImageNet model
# base_model = keras.applications.VGG16(
#     weights='imagenet',  
#     input_shape=(224, 224, 3),
#     include_top=False)

model = models.Sequential()
model.add(layers.Conv2D(56, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(112, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(112, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(224, activation='relu'))
model.add(layers.Dense(2, activation = 'softmax'))


model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 56)      1568      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 56)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 112)     56560     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 112)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 112)       113008    
                                                                 
 flatten (Flatten)           (None, 302848)            0

In [4]:
def save_loss_plot(history):
    plt.plot(history['accuracy'], label='Training Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Fruit Classification, Batch Size: ' + batch_size + ' Epochs: ' + epochs)
    plt.legend()
    plt.savefig(main_dir + '/' + 'model_b' + batch_size + '_e' + epochs + '.png')

In [5]:
# Now it's time to compile the model with loss and metrics options. 
model.compile(optimizer='Adam', loss = 'categorical_crossentropy' , metrics = ['accuracy'])

datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0, # randomly zoom image 
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False) # randomly flip images

# These are data augmentation steps
if(augment_data.lower() in ['true', '1', 't', 'y', 'yes']):
    datagen = ImageDataGenerator(
            samplewise_center=True,  # set each sample mean to 0
            rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.05, # randomly zoom image 
            width_shift_range=0.05,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.05,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False) # randomly flip images

# load and iterate training dataset
train_it = datagen.flow_from_directory( data + '/train/', 
                                       target_size=(224,224), 
                                       color_mode='rgb', 
                                       batch_size=my_batch_size,
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory( data + '/test/', 
                                      target_size=(224,224), 
                                      color_mode='rgb', 
                                       batch_size=my_batch_size,
                                      class_mode="categorical")



Found 887 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [6]:
# Train the model
history_object = model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=my_epochs,
          verbose=1)

if(fine_tune.lower() in ['true', '1', 't', 'y', 'yes']):
    # This will improve the accuracy of the model by fine tuning the training on the entire unfrozen model.  
    # Unfreeze the base model
    base_model.trainable = True
    # Compile the model with a low learning rate
    model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
                  loss =  'categorical_crossentropy' , metrics = ['accuracy'])

    history_object = model.fit(train_it,
              validation_data=valid_it,
              steps_per_epoch=train_it.samples/train_it.batch_size,
              validation_steps=valid_it.samples/valid_it.batch_size,
              epochs=my_epochs, verbose=2)


Epoch 1/80
22/22 [==============================] - 94s 4s/step - loss: 369.2764 - accuracy: 0.5411 - val_loss: 0.6721 - val_accuracy: 0.6250
Epoch 2/80
 5/22 [=====>........................] - ETA: 1:13 - loss: 0.6765 - accuracy: 0.5700

KeyboardInterrupt: 

In [ ]:

save_loss_plot(history_object.history)
model.save(main_dir + '/' + h5modeloutput)